In [3]:
import pandas as pd
import tensorflow as tf
from pathlib import Path
from sklearn.model_selection import train_test_split

In [4]:
basic = tf.keras.models.load_model("models/basic_model.h5")

In [5]:
def load_opt_models(num):
    ret = list()
    search_str = "models/optimized_model_"
    end_str = ".h5"
    for i in range(1, num+1):
        ret.append(tf.keras.models.load_model(search_str + str(i) + end_str))
    return ret

In [6]:
num_opt_models = 4
opt_models = load_opt_models(num_opt_models)

all_models = {"basic": basic}
for i in range(len(opt_models)):
    all_models["opt" + str(i+1)] = opt_models[i]

for model in all_models.values():
    print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                80        
                                                                 
 dense_1 (Dense)             (None, 20)                220       
                                                                 
 dense_2 (Dense)             (None, 5)                 105       
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 411
Trainable params: 411
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (N

In [7]:
def transform_input(sqft, bed, bath, loc, year):
    ret = [sqft, bed, bath, year]
    if loc.lower() == "rural":
        ret += [1, 0, 0]
    elif loc.lower() == "suburban":
        ret += [0, 1, 0]
    elif loc.lower() == "urban":
        ret += [0, 0, 1]
    else:
        return []
    return ret

In [8]:
inp = transform_input(sqft = 2500, bed = 4, bath = 1, year = 2000, loc = "urban")

for name, model in all_models.items():
    value = model.predict([inp])
    print(f"{name} predicted a value of {value[0][0]}")

1/1 [==============================] - 0s 228ms/step
basic predicted a value of 275349.625
1/1 [==============================] - 0s 124ms/step
opt1 predicted a value of 271323.6875
1/1 [==============================] - 0s 125ms/step
opt2 predicted a value of 269215.71875
1/1 [==============================] - 0s 121ms/step
opt3 predicted a value of 272169.8125
1/1 [==============================] - 0s 142ms/step
opt4 predicted a value of 270016.3125


In [9]:
def set_up_data():
    df = pd.read_csv(Path("../data/housing_price_dataset.csv"))
    full_df = pd.get_dummies(df)
    X = full_df.drop(["Price"], axis=1)
    y = full_df[["Price"]]
    return train_test_split(X, y, random_state = 50)

In [10]:
X_train, X_test, y_train, y_test = set_up_data()
for name, model in all_models.items():
    print(f"FOR MODEL {name}")
    model_loss, model_msle = model.evaluate(X_test, y_test, verbose=2)

FOR MODEL basic
391/391 - 1s - loss: 0.1348 - msle: 0.1348 - 955ms/epoch - 2ms/step
FOR MODEL opt1
391/391 - 1s - loss: 0.1345 - msle: 0.1345 - 665ms/epoch - 2ms/step
FOR MODEL opt2
391/391 - 1s - loss: 0.1345 - msle: 0.1345 - 648ms/epoch - 2ms/step
FOR MODEL opt3
391/391 - 1s - loss: 0.1342 - msle: 0.1342 - 657ms/epoch - 2ms/step
FOR MODEL opt4
391/391 - 1s - loss: 0.1342 - msle: 0.1342 - 658ms/epoch - 2ms/step
